### Notice!!! Please use search function to check all the cells with "notice" keywords, those are sanity checks and potential manual variable changes

In [0]:
#########################################################################################################
# Please define the target Timestamp for the end date for the generation of COVID patients dataset
# Notice: Format: YYYY-MM-DD
# Current set to 2021-12-10
############################################################################################################
# check to make sure they are consistent
# Previous dates used:
# 1. "2021-12-31", "20211231"
# 2. "2022-02-18", "20220218"
# 3. end_date, file_date = "2021-12-25", "20211225"
# end_date, file_date = "2021-12-21", "20211221"

start_date, end_date, file_date = "2020-03-01", "2021-12-25", "20211225_Lancet_bmi"
# register it so can be directly used in the following SQL statement
spark.conf.set("enddate.var", end_date)

In [0]:
#pip install xgboost
#pip install joblib
#pip install scikit-learn
#pip install shap

### Keep the follow cell empty can copy all the install commands and run them one by one

In [0]:
import numpy as np
import pandas as pd
import shap

## Scikit-learn related
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import MultiLabelBinarizer
# from sklearn.linear_model import SGDClassifier
# from sklearn.neural_network import MLPClassifier
from sklearn import tree
from sklearn.ensemble import BaggingClassifier

from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
# from sklearn.neighbors import KNeighborsClassifier
## from sklearn import cross_validation, metrics   #Additional scklearn functions, deprecated since version 0.18
## from sklearn.grid_search import GridSearchCV   #!!!the grid search package that has issue, dont use it
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

from sklearn.metrics import roc_curve, auc, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import LabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from scipy import interp
from scipy import stats

from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing

########################################
## Import those from pyspark.ml
import pyspark.ml
import pyspark.sql.functions as F

In [0]:
imids_training_df = spark.sql("""SELECT * FROM rdp_phi_sandbox.qw_IMID_COVID_trainset_cond_med_vax_{}""".format(file_date)) 
# imids_training_df.limit(5).toPandas()

In [0]:
# imids_training_df.limit(20).toPandas()

In [0]:
imids_training_df = imids_training_df.dropDuplicates()
# imids_training_df.limit(5).toPandas()

In [0]:
## Drop not used columns
col_to_drop = ('age_range', 'ethnicity', 'race1', 'race_v2', 'ethnicity_race', 'CVX_name', 'decided_index_date', 'CVX_name')
imids_training_df = imids_training_df.drop(*col_to_drop)

## rename IMIDs drugs columns
imids_training_df = imids_training_df.withColumnRenamed('prior_91_days_hydroxychloroquine_logic', 'hydroxychloroquine').withColumnRenamed('prior_91_days_methotrexate_logic', 'methotrexate')\
.withColumnRenamed('prior_91_days_leflunomide_teriflunomide_logic', 'leflunomide_teriflunomide').withColumnRenamed('prior_91_days_5_ASAa_logic', '5_ASA')\
.withColumnRenamed('prior_91_days_azathioprine_logic', 'azathioprine').withColumnRenamed('prior_91_days_mercaptopurine_logic', 'mercaptopurine')\
.withColumnRenamed('prior_91_days_mitoxantrone_logic', 'mitoxantrone')\
.withColumnRenamed('prior_91_days_mycophenolate_logic', 'mycophenolate').withColumnRenamed('prior_91_days_calcineurin_inhibitor_logic', 'calcineurin_inhibitor')\
.withColumnRenamed('prior_91_days_TNF_alpha_inhibitor_logic', 'TNF_alpha_inhibitor').withColumnRenamed('prior_91_days_fumarates_logic', 'fumarates')\
.withColumnRenamed('prior_91_days_interferons_logic', 'interferons').withColumnRenamed('prior_91_days_alkylating_agent_logic', 'alkylating_agent')\
.withColumnRenamed('prior_91_days_hydroxyurea_logic', 'hydroxyurea').withColumnRenamed('prior_91_days_dapsone_logic', 'dapsone')\
.withColumnRenamed('prior_91_days_cladribine_logic', 'cladribine').withColumnRenamed('prior_91_days_IL1_inhibitor_logic', 'IL1_inhibitor')\
.withColumnRenamed('prior_91_days_IL6_inhibitor_logic', 'IL6_inhibitor').withColumnRenamed('prior_91_days_IL12_23_inhibitor_logic', 'IL12_23_inhibitor')\
.withColumnRenamed('prior_91_days_IL17_inhibitor_logic', 'IL17_inhibitor')\
.withColumnRenamed('prior_91_days_IL23_inhibitor_logic', 'IL23_inhibitor')\
.withColumnRenamed('prior_91_days_abatacept_logic', 'abatacept').withColumnRenamed('prior_91_days_anti_BLyS_logic', 'anti_BLyS')\
.withColumnRenamed('prior_91_days_S1P_receptor_modulator_logic', 'S1P_receptor_modulator').withColumnRenamed('prior_91_days_JAK_inhibitor_logic', 'JAK_inhibitor')\
.withColumnRenamed('prior_91_days_integrin_inhibitor_logic', 'integrin_inhibitor').withColumnRenamed('prior_91_days_PDE4i_targeted_synthetic_logic', 'PDE4i_targeted_synthetic')\
.withColumnRenamed('prior_91_days_anti_CD20_logic', 'anti_CD20').withColumnRenamed('prior_91_days_anti_CD52_logic', 'anti_CD52')\
.withColumnRenamed('prior_91_days_budesonide_logic', 'budesonide').withColumnRenamed('prior_91_days_systemic_glucocorticoids_logic', 'systemic_glucocorticoids')\
.withColumnRenamed('after_10_days_monoclonal_antibody_covid_19_logic', 'monoclonal_antibody_covid_19')

In [0]:
## Temp solution, filter the positive patient here
pos_df = imids_training_df.filter(imids_training_df['results'] == "Positive")

## convert to pandas dataframe
# imids_training_pd_df = imids_training_onlyPos_df.toPandas()
 
## col names before dropping any
# list(imids_training_pd_df.columns)

In [0]:
## One hot-encode sex
categ = pos_df.select('sex').distinct().rdd.flatMap(lambda x:x).collect()
exprs = [F.when(F.col('sex') == cat,1).otherwise(0)\
            .alias(str("sex_" + cat)) for cat in categ]
pos_one_hot_df = pos_df.select(pos_df.columns + exprs)

## Fill none values in the vaccination column
pos_one_hot_df = pos_one_hot_df.fillna({'Vaccination_status':'Not'})

## One hot-encode vaccination
categ = pos_one_hot_df.select('Vaccination_status').distinct().rdd.flatMap(lambda x:x).collect()
exprs = [F.when(F.col('Vaccination_status') == cat,1).otherwise(0)\
            .alias(str("vaccination_" + cat)) for cat in categ]
pos_one_hot_df = pos_one_hot_df.select(pos_one_hot_df.columns + exprs)

## Drop not needed cols and original cols
pos_one_hot_df = pos_one_hot_df.drop("sex").drop("sex_Female").drop("sex_Unknown").drop("Vaccination_status").drop("vaccination_Not")

## change the name of those vaccination related columns
pos_one_hot_df = pos_one_hot_df.withColumnRenamed('vaccination_Fully', 'fully_vaccinated').withColumnRenamed('vaccination_Booster', 'boosted')

In [0]:
# ## Manually define the min max process
# from pyspark.sql.functions import max, min, mean, col

# max_age, min_age = pos_one_hot_df.select(max("age"), min("age")).first()
# pos_one_hot_minmax_df = pos_one_hot_df.withColumn("age_normalized", (col("age") - min_age) / (max_age - min_age) )
# pos_one_hot_minmax_df = pos_one_hot_minmax_df.drop("age")

In [0]:
# ## get the data where monoclonal_antibody_covid_19 = 0
# pos_one_hot_noAntibody_df = pos_one_hot_df.where(pos_one_hot_df.monoclonal_antibody_covid_19 == 0)

In [0]:
train_df = pos_one_hot_df.select("*")
train_df = train_df.drop("pat_id").drop("results")

## Drop SVI and geocoding features due to high missing percentage
## list: 'SVI_Socioeconomic', 'SVI_Household_Composition_Disability', 'SVI_Minority_Status_Language', 'SVI_Housing_Type_Transportation', 'SVI', 'Metro_area', 'Low_education', 'Low_employment',
## not including 'obesity'

train_df = train_df.select('patient_id', 'hospitalized_after_positive','IMV_after_positive','death_after_positive',
  'age', 'BMI', 'sex_Male',
  'hypertension', 'diabetes_type1and2', 'atrial_fibrillation', 'coronary_artery_disease', 'heart_failure', 'chronic_kidney_disease', 'copd', 'chronic_liver_disease', 'malignant_neoplastic_disease', 'asthma', 'HIV', 'history_transplant', 'stroke', 'opioid_dependence', 'fully_vaccinated', 'boosted', 
 'ibd', 'rheumatoid_arthritis', 'multiple_sclerosis','psoriatic_arthritis', 'psoriasis', 'systemic_sclerosis', 'spondyloarthritis', 'systemic_lupus', 'vasculitis', 'sarcoidosis', 'APS', 'sjogren_syndrome',
 'hydroxychloroquine',
 'methotrexate',
 'leflunomide_teriflunomide',
 '5_ASA',
 'azathioprine',
 'mercaptopurine',
 'mitoxantrone',
 'mycophenolate',
 'calcineurin_inhibitor',
 'TNF_alpha_inhibitor',
 'fumarates',
 'interferons',
 'alkylating_agent',
 'hydroxyurea',
 'dapsone',
 'cladribine',
 'IL1_inhibitor',
 'IL6_inhibitor',
 'IL12_23_inhibitor',
 'IL17_inhibitor',
 'IL23_inhibitor',
 'abatacept',
 'anti_BLyS',
 'S1P_receptor_modulator',
 'JAK_inhibitor',
 'integrin_inhibitor',
 'PDE4i_targeted_synthetic',
 'anti_CD20',
 'anti_CD52',
 'budesonide',
 'systemic_glucocorticoids',
 'monoclonal_antibody_covid_19')

In [0]:
train_pd_df = train_df.toPandas()
## change the patient id col to be the index of the training data
train_pd_df = train_pd_df.set_index("patient_id")
display(train_pd_df)

iteritems is deprecated and will be removed in a future version. Use .items instead.

hospitalized_after_positive,IMV_after_positive,death_after_positive,age,BMI,sex_Male,hypertension,diabetes_type1and2,atrial_fibrillation,coronary_artery_disease,heart_failure,chronic_kidney_disease,copd,chronic_liver_disease,malignant_neoplastic_disease,asthma,HIV,history_transplant,stroke,opioid_dependence,fully_vaccinated,boosted,ibd,rheumatoid_arthritis,multiple_sclerosis,psoriatic_arthritis,psoriasis,systemic_sclerosis,spondyloarthritis,systemic_lupus,vasculitis,sarcoidosis,APS,sjogren_syndrome,hydroxychloroquine,methotrexate,leflunomide_teriflunomide,5_ASA,azathioprine,mercaptopurine,mitoxantrone,mycophenolate,calcineurin_inhibitor,TNF_alpha_inhibitor,fumarates,interferons,alkylating_agent,hydroxyurea,dapsone,cladribine,IL1_inhibitor,IL6_inhibitor,IL12_23_inhibitor,IL17_inhibitor,IL23_inhibitor,abatacept,anti_BLyS,S1P_receptor_modulator,JAK_inhibitor,integrin_inhibitor,PDE4i_targeted_synthetic,anti_CD20,anti_CD52,budesonide,systemic_glucocorticoids,monoclonal_antibody_covid_19
0,0,0,17.0,27.284075066537667,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,36.0,34.0892345261253,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,29.0,24.902644681653808,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,56.0,24.96102980371641,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,20.0,28.500801971148157,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,66.0,27.435054870109738,0,1,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,30.0,25.793064130506956,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,36.0,32.76493765597761,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,66.0,29.345224520898874,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,76.0,30.865702475349593,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Training data preprocessing and test data split
* Assign features for training data to X
* Assign response to Y
* Define the number of CV & random seed used for tuning

In [0]:
# ### Uncomment to save table as backup for publications
# spark.sql("""DROP TABLE IF EXISTS rdp_phi_sandbox.qw_IMIDs_COVID_paper_train_data_processed_{}""".format(file_date))
# table_name = "rdp_phi_sandbox.qw_IMIDs_COVID_paper_train_data_processed_{}".format(file_date)
# ## Convert into Spark DataFrame
# spark_df_one_hot_encoded = spark.createDataFrame(df_one_hot_encoded)

# ## Write the table
# # spark_df_one_hot_encoded.write.mode("overwrite").saveAsTable(table_name)

In [0]:
#######################################################
#Random search CV method
#and
#Multi class roc_auc score method
########################################################
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
from time import time
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import make_scorer, roc_auc_score

###############################################################################################
#Binary class roc auc score method
#input: y_true, true labels from test fold
#       y_score, predicted probability on test fold
#       average, string, [None, ‘micro’, ‘macro’ (default), ‘samples’, ‘weighted’]
#                'macro': Calculate metrics for each label, and find their unweighted mean. 
#                This does not take label imbalance into account.
#                'weighted': Calculate metrics for each label, and find their average, 
#                weighted by support (the number of true instances for each label).
#output: auroc value for each class
#############################################################################################
def binary_class_roc_auc_score(y_true, y_score, average="weighted"):

    return roc_auc_score(y_true, y_score, average=average)

binaryclass_score = make_scorer(binary_class_roc_auc_score, needs_threshold = True)

In [0]:
## not including: , 'obesity'
train_pd_df.columns = ['hospitalized_after_positive', 'IMV_after_positive', 'death_after_positive', 'age', 'BMI', 'sex:male', 'hypertension', 'diabetes (type 1+2)', 'atrial fibrillation', 'coronary artery disease', 'heart failure', 'chronic kidney disease', 'COPD', 'chronic liver disease', 'malignant neoplastic disease', 'asthma', 'HIV', 'history of transplant', 'stroke', 'opioid dependence', 'fully vaccinated', 'boosted', 
 'inflammatory bowel disease', 'rheumatoid arthritis', 'multiple sclerosis','psoriatic arthritis', 'psoriasis', 'systemic sclerosis', 'spondyloarthritis', 'systemic lupus', 'vasculitis', 'sarcoidosis', 'antiphospholipid syndrome', 'Sjögren syndrome',
 'hydroxychloroquine',
 'methotrexate',
 'leflunomide teriflunomide',
 '5-ASA',
 'azathioprine',
 'mercaptopurine',
 'mitoxantrone',
 'mycophenolate',
 'calcineurin inhibitor',
 'TNF-α inhibitor',
 'fumarates',
 'interferons',
 'alkylating agent',
 'hydroxyurea',
 'dapsone',
 'cladribine',
 'IL-1 inhibitor',
 'IL-6 inhibitor',
 'IL-12/23 inhibitor',
 'IL-17 inhibitor',
 'IL-23 inhibitor',
 'abatacept',
 'anti-BLyS',
 'S1P receptor modulator',
 'JAK inhibitor',
 'integrin inhibitor',
 'PDE4i targeted synthetic',
 'anti-CD20',
 'anti-CD52',
 'budesonide',
 'systemic glucocorticoids', 'monoclonal antibody covid-19']

In [0]:
## Columns to drop based on VIF results or missingness
# train_pd_df.drop(['HIV'], axis = 1, inplace = True)
# train_pd_df.drop(['systemic sclerosis'], axis = 1, inplace = True)
# train_pd_df.drop(['leflunomide teriflunomide'], axis = 1, inplace = True)
# train_pd_df.drop(['5-ASA'], axis = 1, inplace = True)
train_pd_df.drop(['mitoxantrone'], axis = 1, inplace = True)
# train_pd_df.drop(['mycophenolate'], axis = 1, inplace = True)
# train_pd_df.drop(['interferons'], axis = 1, inplace = True)
# train_pd_df.drop(['alkylating agent'], axis = 1, inplace = True)
# train_pd_df.drop(['hydroxyurea'], axis = 1, inplace = True)

# train_pd_df.drop(['dapsone'], axis = 1, inplace = True)
train_pd_df.drop(['cladribine'], axis = 1, inplace = True)
train_pd_df.drop(['IL-1 inhibitor'], axis = 1, inplace = True)
# train_pd_df.drop(['IL-6 inhibitor'], axis = 1, inplace = True)
# train_pd_df.drop(['IL-23 inhibitor'], axis = 1, inplace = True)
train_pd_df.drop(['anti-BLyS'], axis = 1, inplace = True)
# train_pd_df.drop(['S1P receptor modulator'], axis = 1, inplace = True)
train_pd_df.drop(['integrin inhibitor'], axis = 1, inplace = True)
# train_pd_df.drop(['PDE4i targeted synthetic'], axis = 1, inplace = True)
# train_pd_df.drop(['anti-CD20'], axis = 1, inplace = True)
train_pd_df.drop(['anti-CD52'], axis = 1, inplace = True)

# train_pd_df.drop(['mercaptopurine'], axis = 1, inplace = True)
# train_pd_df.drop(['abatacept'], axis = 1, inplace = True)

# train_pd_df.drop(['IL-1 inhibitor'], axis = 1, inplace = True)
# train_pd_df.drop(['cladribine'], axis = 1, inplace = True)
# train_pd_df.drop(['anti-CD52'], axis = 1, inplace = True)
# train_pd_df.drop(['monoclonal antibody covid-19'], axis = 1, inplace = True)


percent_missing = train_pd_df.isnull().sum() * 100 / len(train_pd_df)
missing_value_df = pd.DataFrame({'column_name': train_pd_df.columns,
                                 'percent_missing': percent_missing})

display(missing_value_df)

## Notice! Need to comment out once have the initial VIF results
## Fill NA values
## Method 1: Remove rows with at least one null value
# train_pd_noNull_df = train_pd_df.dropna()
# train_pd_df = train_pd_noNull_df

## Notice
## Remove mAbs, antibody feature here
train_pd_df.drop(['monoclonal antibody covid-19'], axis = 1, inplace = True)

iteritems is deprecated and will be removed in a future version. Use .items instead.

column_name,percent_missing
hospitalized_after_positive,0.0
IMV_after_positive,0.0
death_after_positive,0.0
age,0.0
BMI,0.0
sex:male,0.0
hypertension,0.0
diabetes (type 1+2),0.0
atrial fibrillation,0.0
coronary artery disease,0.0


In [0]:
## manually set the random seed to define a replication
r_seed = 42
print("current random seed is: ", r_seed)

## manually set the number for cross validation
num_cv = 10
print("current CV fold selection is: ", num_cv)

## Possible response vectors
Y_cols = ["hospitalized_after_positive", 'IMV_after_positive', 'death_after_positive']

## separate X and Y
train_df_Y = train_pd_df[Y_cols]
train_df_X = train_pd_df.drop(Y_cols, axis=1)

## Create composite response vectors
train_df_Y_new = pd.DataFrame(train_df_Y, columns = Y_cols)

train_df_Y_new['hospitalized_or_IMV_or_death'] = train_df_Y['hospitalized_after_positive'] + train_df_Y['IMV_after_positive'] + train_df_Y['death_after_positive']
train_df_Y_new['IMV_or_death'] = train_df_Y['IMV_after_positive'] + train_df_Y['death_after_positive']

## Convert those >1 values back to 1
train_df_Y_new.loc[train_df_Y_new['hospitalized_or_IMV_or_death'] >= 1, 'hospitalized_or_IMV_or_death'] = 1
train_df_Y_new.loc[train_df_Y_new['IMV_or_death'] >= 1, 'IMV_or_death'] = 1

current random seed is: 42
current CV fold selection is: 10

In [0]:
## Notice: LR need a minimum of 10 events (>50 better) per independent variable
## Therefore, the following features need to be excluded for lack of events

# train_df_X.iloc[:,2:].apply(pd.Series.value_counts)

In [0]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = train_df_X.columns

# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(train_df_X.values, i) for i in range(len(train_df_X.columns))]

display(vif_data)

iteritems is deprecated and will be removed in a future version. Use .items instead.

feature,VIF
age,2.986029434453344
BMI,1.0044149158264668
sex:male,1.6477237618579097
hypertension,1.7242291659273654
diabetes (type 1+2),1.3578634344339255
atrial fibrillation,1.2958755059183693
coronary artery disease,1.2985371762335238
heart failure,1.4445068264844114
chronic kidney disease,1.3509897567196825
COPD,1.1869409265589888


In [0]:
# import matplotlib.pyplot as plt
# from bioinfokit import visuz

# visuz.stat.corr_mat(df=train_df_X, cmap='RdBu',dim = (15,15),  show=True)

In [0]:
## Found workaround at blog: this https://stackoverflow.com/questions/71106940/cannot-import-name-centered-from-scipy-signal-signaltools
import  scipy.signal.signaltools

def _centered(arr, newsize):
    # Return the center newsize portion of the array.
    newsize = np.asarray(newsize)
    currsize = np.array(arr.shape)
    startind = (currsize - newsize) // 2
    endind = startind + newsize
    myslice = [slice(startind[k], endind[k]) for k in range(len(endind))]
    return arr[tuple(myslice)]

scipy.signal.signaltools._centered = _centered

In [0]:
import statsmodels.api as sm
import statsmodels
import lxml

## The regularization method AND the solver used is determined by the argument method.
regular_method = "l1"

##Set LR optimzer to be
## Options: lbfgs, powell, cg, ncg, bfgs, basinhopping, newton, ‘nm’ for Nelder-Mead
## Previous choice: ncg
opt_method = 'lbfgs'

####################################################################################################
## Method used to do univariate logistic regression on each single feature/variable
## Inputs:
## os_X: oversampled training data set X
## os_y: oversampled training data's label y
## X: training data set X
## y: training data's label y
## variable_name: the current feature/variable to use for the univariate LR model
## opt_method: the current approximation method used for the LR model
## Output:
## A dataframe with columns: Feature, Feature Importance, Odds ratio, Pvalue, lower, upper
######################################################################################################
def multivariate_LR_model_result(os_X, os_y, us_X, us_y, select_method):
  ## Instantiate a bionomial family and logit link function GLM model as logistic regression model
  os_model = sm.Logit(os_y, os_X, missing = 'drop').fit(method = select_method, skip_hessian = False)

  # print(LR_statmodel.summary())

  ## Acquire LR summary
  os_results_summary = os_model.summary()
  ## Note that tables is a list. The table at index 1 is the "core" table. Additionally, read_html puts dfs in a list, so we want index 0
  os_results_as_html = os_results_summary.tables[1].as_html()
  os_odds_ratio_df = pd.read_html(os_results_as_html, header=0, index_col=0)[0]

    ## get the CI interval
  os_odds_ratio_df = os_odds_ratio_df[["coef", "[0.025", "0.975]"]]
  os_odds_ratio_df.rename(columns={'coef': 'Odds ratio', '[0.025': 'lower', '0.975]':'upper'}, inplace=True)

  ## Instantiate a bionomial family and logit link function GLM model as logistic regression model
  us_model = sm.Logit(us_y, us_X, missing = 'drop').fit(method = select_method, skip_hessian = False)

  # print(LR_statmodel.summary())

  ## Acquire LR summary
  us_results_summary = us_model.summary()
  ## Note that tables is a list. The table at index 1 is the "core" table. Additionally, read_html puts dfs in a list, so we want index 0
  us_results_as_html = us_results_summary.tables[1].as_html()
  us_odds_ratio_df = pd.read_html(us_results_as_html, header=0, index_col=0)[0]

  ## Get the uncorrected P value
  us_odds_ratio_df = us_odds_ratio_df[["P>|z|"]]
  us_odds_ratio_df.rename(columns={'P>|z|': 'Pvalue'}, inplace=True)
  
  ## Get the corrected P value
  ## Now use bonferroni
  ## check this document for more details: http://jpktd.blogspot.com/2013/04/multiple-testing-p-value-corrections-in.html
  ## API: https://www.statsmodels.org/dev/generated/statsmodels.stats.multitest.multipletests.html
  ## Options: conservative Bonferroni correction method
  rej, pval_corr= statsmodels.stats.multitest.multipletests(us_odds_ratio_df['Pvalue'], alpha=0.1, method='fdr_by', is_sorted=False)[:2]
  us_odds_ratio_df['Pvalue_FDR_corrected'] = pval_corr

  odds_ratio_df = pd.concat([os_odds_ratio_df.reset_index(drop=True), us_odds_ratio_df.reset_index(drop=True)], axis=1)

  ## Display the odds ratio, p value, feature importance dataframe
  feature_importances = pd.DataFrame(os_model.conf_int()[1]).rename(columns={1:'Coefficients'}).eval("absolute_coefficients=abs(Coefficients)")
  feature_importances.reset_index(inplace=True)

  ## if feature importance is needed
  feature_importances_df = feature_importances[["index", "absolute_coefficients"]]
  feature_importances_df.rename(columns={'index': 'Feature', 'absolute_coefficients': 'Feature Importance'}, inplace=True)
  ## if feature importance is not needed
#   feature_importances_df = feature_importances[["index"]]
#   feature_importances_df.rename(columns={'index': 'Feature'}, inplace=True)

  result_df = pd.concat([feature_importances_df.reset_index(drop=True), odds_ratio_df.reset_index(drop=True)], axis=1)
  return result_df, os_model

## Response == "hospitalized_after_positive"

In [0]:
# from imblearn.over_sampling import RandomOverSampler
# from imblearn.under_sampling import RandomUnderSampler

# ###############################################################################
# ## Current Y as response
# ## Notice: need manual check
# ## Possible options: hospitalized, invasive_mechanical_vent, death, results
# ################################################################################

# select_col = 'hospitalized_after_positive'
# Y = train_df_Y[select_col]
# # Y = Y.map(dict(yes=1, no=0))

# class_names = np.unique(Y)
# print("unique labels from y: ", class_names)

# ## Train test split use r_seed assigned in CMD 1
# X = train_df_X
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=r_seed, stratify = Y)
# columns = X_train.columns

# ## Fill NA after the train/test split to avoid any potential data leakage
# ## Method 2: Impute those null values in each column with the median value of the column
# X_train = X_train.fillna(X_train.mean())
# X_test = X_test.fillna(X_train.mean())


# ## Random oversampling using random seed
# ## Define oversampling strategy: default, 'auto': equivalent to 'not majority'
# oversample = RandomOverSampler(random_state=r_seed)
# os_data_X, os_data_y = oversample.fit_resample(X_train, y_train)
# os_data_X = pd.DataFrame(data=os_data_X, columns=columns)
# os_data_y= pd.DataFrame(data=os_data_y, columns=['hospitalized_after_positive'])


# # define undersample strategy: “majority” will undersample the majority class determined by the class with the largest number of examples.
# # undersample = RandomUnderSampler(sampling_strategy='majority', random_state=r_seed)
# # us_data_X, us_data_y = undersample.fit_resample(X_train, y_train)
# # us_data_X = pd.DataFrame(data=us_data_X, columns=columns)
# # us_data_y= pd.DataFrame(data=us_data_y, columns=['hospitalized_after_positive'])


# # we can Check the numbers of our data
# # print("length of oversampled data is ",len(os_data_X))
# # print("Number of no hos in oversampled data",len(os_data_y[os_data_y['hospitalized_after_positive']==0]))
# # print("Number of hos",len(os_data_y[os_data_y['hospitalized_after_positive']==1]))
# # print("Proportion of no hos data in oversampled data is ",len(os_data_y[os_data_y['hospitalized_after_positive']==0])/len(os_data_X))
# # print("Proportion of hos data in oversampled data is ",len(os_data_y[os_data_y['hospitalized_after_positive']==1])/len(os_data_X))

In [0]:
## Notice: LR need a minimum of 10 events (>50 better) per independent variable
## Therefore, the following features need to be excluded for lack of events

# us_data_X.iloc[:,2:].apply(pd.Series.value_counts)

In [0]:
# hos_result_df, os_LR_statmodel = multivariate_LR_model_result(os_data_X, os_data_y, X_train, y_train, opt_method)
# display(hos_result_df)

In [0]:
# from sklearn.metrics import roc_auc_score
# LR_model_binaryclass_auroc = binary_class_roc_auc_score(y_test, os_LR_statmodel.predict(X_test))

# print("Auroc on test data set: %0.5f" % (LR_model_binaryclass_auroc))

## Response == "invasive_mechanical_vent"

In [0]:
# from imblearn.over_sampling import RandomOverSampler
# from imblearn.under_sampling import RandomUnderSampler

# ###############################################################################
# ## Current Y as response
# ## Notice: need manual check
# ## Possible options: hospitalized, invasive_mechanical_vent, death, results
# ################################################################################

# select_col = 'IMV_after_positive'
# Y = train_df_Y[select_col]
# # Y = Y.map(dict(yes=1, no=0))

# class_names = np.unique(Y)
# print("unique labels from y: ", class_names)

# ## Train test split use r_seed assigned in CMD 1
# X = train_df_X
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=r_seed, stratify = Y)
# columns = X_train.columns

# ## Fill NA after the train/test split to avoid any potential data leakage
# ## Method 2: Impute those null values in each column with the median value of the column
# X_train = X_train.fillna(X_train.mean())
# X_test = X_test.fillna(X_train.mean())

# ## Random oversampling using random seed
# ## Define oversampling strategy: default, 'auto': equivalent to 'not majority'
# oversample = RandomOverSampler(random_state=r_seed)
# os_data_X, os_data_y = oversample.fit_resample(X_train, y_train)
# os_data_X = pd.DataFrame(data=os_data_X, columns=columns)
# os_data_y= pd.DataFrame(data=os_data_y, columns=['IMV_after_positive'])


# # define undersample strategy: “majority” will undersample the majority class determined by the class with the largest number of examples.
# # undersample = RandomUnderSampler(sampling_strategy='majority', random_state=r_seed)
# # us_data_X, us_data_y = undersample.fit_resample(X_train, y_train)
# # us_data_X = pd.DataFrame(data=us_data_X, columns=columns)
# # us_data_y= pd.DataFrame(data=us_data_y, columns=['IMV_after_positive'])


# # we can Check the numbers of our data
# # print("length of oversampled data is ",len(os_data_X))
# # print("Number of no hos in oversampled data",len(os_data_y[os_data_y['hospitalized_after_positive']==0]))
# # print("Number of hos",len(os_data_y[os_data_y['hospitalized_after_positive']==1]))
# # print("Proportion of no hos data in oversampled data is ",len(os_data_y[os_data_y['hospitalized_after_positive']==0])/len(os_data_X))
# # print("Proportion of hos data in oversampled data is ",len(os_data_y[os_data_y['hospitalized_after_positive']==1])/len(os_data_X))

In [0]:
# imv_result_df, os_LR_statmodel = multivariate_LR_model_result(os_data_X, os_data_y, X_train, y_train, opt_method)
# display(imv_result_df)

In [0]:
# from sklearn.metrics import roc_auc_score
# LR_model_binaryclass_auroc = binary_class_roc_auc_score(y_test, os_LR_statmodel.predict(X_test))

# print("Auroc on test data set: %0.5f" % (LR_model_binaryclass_auroc))

## Response == "death"

In [0]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

###############################################################################
## Current Y as response
## Notice: need manual check
## Possible options: hospitalized, invasive_mechanical_vent, death, results
################################################################################

select_col = 'death_after_positive'
Y = train_df_Y[select_col]
# Y = Y.map(dict(yes=1, no=0))

class_names = np.unique(Y)
print("unique labels from y: ", class_names)

## Train test split use r_seed assigned in CMD 1
X = train_df_X
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=r_seed, stratify = Y)
columns = X_train.columns

## Normalize after the train/test split to avoid any potential data leakage
## using the min and max from the trainset to do minmax on test set
age_min, age_max = X_train["age"].min(), X_train["age"].max()
X_train["age"] = (X_train["age"] - age_min) / (age_max - age_min)
X_test["age"] = (X_test["age"] - age_min) / (age_max - age_min)

BMI_min, BMI_max = X_train["BMI"].min(), X_train["BMI"].max()
X_train["BMI"] = (X_train["BMI"] - BMI_min) / (BMI_max - BMI_min)
X_test["BMI"] = (X_test["BMI"] - BMI_min) / (BMI_max - BMI_min)

## Random oversampling using random seed
## Define oversampling strategy: default, 'auto': equivalent to 'not majority'
oversample = RandomOverSampler(random_state=r_seed)
os_data_X, os_data_y = oversample.fit_resample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X, columns=columns)
os_data_y= pd.DataFrame(data=os_data_y, columns=['death_after_positive'])


# define undersample strategy: “majority” will undersample the majority class determined by the class with the largest number of examples.
# undersample = RandomUnderSampler(sampling_strategy='majority', random_state=r_seed)
# us_data_X, us_data_y = undersample.fit_resample(X_train, y_train)
# us_data_X = pd.DataFrame(data=us_data_X, columns=columns)
# us_data_y= pd.DataFrame(data=us_data_y, columns=['death_after_positive'])


# we can Check the numbers of our data
# print("length of oversampled data is ",len(os_data_X))
# print("Number of no hos in oversampled data",len(os_data_y[os_data_y['hospitalized_after_positive']==0]))
# print("Number of hos",len(os_data_y[os_data_y['hospitalized_after_positive']==1]))
# print("Proportion of no hos data in oversampled data is ",len(os_data_y[os_data_y['hospitalized_after_positive']==0])/len(os_data_X))
# print("Proportion of hos data in oversampled data is ",len(os_data_y[os_data_y['hospitalized_after_positive']==1])/len(os_data_X))

unique labels from y: [0 1]

In [0]:
death_result_df, os_LR_statmodel = multivariate_LR_model_result(os_data_X, os_data_y, X_train, y_train, opt_method)
display(death_result_df)

Maximum Likelihood optimization failed to converge. Check mle_retvals
Maximum Likelihood optimization failed to converge. Check mle_retvals

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
iteritems is deprecated and will be removed in a future version. Use .items instead.

Feature,Feature Importance,Odds ratio,lower,upper,Pvalue,Pvalue_FDR_corrected
age,1.1074176053427618,1.0822,1.057,1.107,0.0,0.0
BMI,4.031920446212336,-0.2402,-4.512,4.032,0.919,1.0
sex:male,0.42641765554037303,-0.4433,-0.46,-0.426,0.0,0.0
hypertension,0.13347854485649877,0.1091,0.085,0.133,0.0,0.0
diabetes (type 1+2),0.3628740755431108,0.3349,0.307,0.363,0.0,0.0
atrial fibrillation,0.7289098092830248,0.6912,0.654,0.729,0.0,0.0
coronary artery disease,0.4604323297963114,0.4231,0.386,0.46,0.0,0.0
heart failure,0.6620697482712743,0.6232,0.584,0.662,0.0,0.0
chronic kidney disease,1.0734023920950104,1.0386,1.004,1.073,0.0,0.0
COPD,0.9075415479108072,0.8664,0.825,0.908,0.0,0.0


In [0]:
from sklearn.metrics import roc_auc_score
LR_model_binaryclass_auroc = binary_class_roc_auc_score(y_test, os_LR_statmodel.predict(X_test))

print("Auroc on test data set: %0.5f" % (LR_model_binaryclass_auroc))

Auroc on test data set: 0.84249

## Response == "hospitalized_or_IMV_or_death"

In [0]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

###############################################################################
## Current Y as response
## Notice: need manual check
## Possible options: hospitalized, invasive_mechanical_vent, death, results
################################################################################

select_col = 'hospitalized_or_IMV_or_death'
Y = train_df_Y_new[select_col]
# Y = Y.map(dict(yes=1, no=0))

class_names = np.unique(Y)
print("unique labels from y: ", class_names)

## Train test split use r_seed assigned in CMD 1
X = train_df_X
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=r_seed, stratify = Y)
columns = X_train.columns

## Normalize after the train/test split to avoid any potential data leakage
## using the min and max from the trainset to do minmax on test set
age_min, age_max = X_train["age"].min(), X_train["age"].max()
X_train["age"] = (X_train["age"] - age_min) / (age_max - age_min)
X_test["age"] = (X_test["age"] - age_min) / (age_max - age_min)

BMI_min, BMI_max = X_train["BMI"].min(), X_train["BMI"].max()
X_train["BMI"] = (X_train["BMI"] - BMI_min) / (BMI_max - BMI_min)
X_test["BMI"] = (X_test["BMI"] - BMI_min) / (BMI_max - BMI_min)

## Random oversampling using random seed
## Define oversampling strategy: default, 'auto': equivalent to 'not majority'
oversample = RandomOverSampler(random_state=r_seed)
os_data_X, os_data_y = oversample.fit_resample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X, columns=columns)
os_data_y= pd.DataFrame(data=os_data_y, columns=['hospitalized_or_IMV_or_death'])


# define undersample strategy: “majority” will undersample the majority class determined by the class with the largest number of examples.
# undersample = RandomUnderSampler(sampling_strategy='majority', random_state=r_seed)
# us_data_X, us_data_y = undersample.fit_resample(X_train, y_train)
# us_data_X = pd.DataFrame(data=us_data_X, columns=columns)
# us_data_y= pd.DataFrame(data=us_data_y, columns=['hospitalized_or_IMV_or_death'])


# we can Check the numbers of our data
# print("length of oversampled data is ",len(os_data_X))
# print("Number of no hos in oversampled data",len(os_data_y[os_data_y['hospitalized_after_positive']==0]))
# print("Number of hos",len(os_data_y[os_data_y['hospitalized_after_positive']==1]))
# print("Proportion of no hos data in oversampled data is ",len(os_data_y[os_data_y['hospitalized_after_positive']==0])/len(os_data_X))
# print("Proportion of hos data in oversampled data is ",len(os_data_y[os_data_y['hospitalized_after_positive']==1])/len(os_data_X))

unique labels from y: [0 1]

In [0]:
hos_IMV_death_result_df, os_LR_statmodel = multivariate_LR_model_result(os_data_X, os_data_y, X_train, y_train, opt_method)
display(hos_IMV_death_result_df)

Maximum Likelihood optimization failed to converge. Check mle_retvals
Maximum Likelihood optimization failed to converge. Check mle_retvals

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
iteritems is deprecated and will be removed in a future version. Use .items instead.

Feature,Feature Importance,Odds ratio,lower,upper,Pvalue,Pvalue_FDR_corrected
age,1.0480438890284418,1.0211,0.994,1.048,0.0,0.0
BMI,3.2085898955753924,-0.1183,-3.445,3.209,0.92,1.0
sex:male,0.2925217038050428,-0.3082,-0.324,-0.293,0.0,0.0
hypertension,0.07927852042382014,-0.1029,-0.126,-0.079,0.0,0.0
diabetes (type 1+2),0.4607935061003422,0.4339,0.407,0.461,0.0,0.0
atrial fibrillation,0.4795184480315094,0.44,0.4,0.48,0.0,0.0
coronary artery disease,0.202329718866199,0.1638,0.125,0.202,0.0,0.0
heart failure,0.5591765726634771,0.5177,0.476,0.559,0.0,0.0
chronic kidney disease,0.6129078923725976,0.5771,0.541,0.613,0.0,0.0
COPD,0.6189591818622585,0.576,0.533,0.619,0.0,0.0


In [0]:
from sklearn.metrics import roc_auc_score
LR_model_binaryclass_auroc = binary_class_roc_auc_score(y_test, os_LR_statmodel.predict(X_test))

print("Auroc on test data set: %0.5f" % (LR_model_binaryclass_auroc))

Auroc on test data set: 0.69951

## Response == "IMV_or_death"

In [0]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

###############################################################################
## Current Y as response
## Notice: need manual check
## Possible options: hospitalized, invasive_mechanical_vent, death, results
################################################################################

select_col = 'IMV_or_death'
Y = train_df_Y_new[select_col]
# Y = Y.map(dict(yes=1, no=0))

class_names = np.unique(Y)
print("unique labels from y: ", class_names)

## Train test split use r_seed assigned in CMD 1
X = train_df_X
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=r_seed, stratify = Y)
columns = X_train.columns

## Normalize after the train/test split to avoid any potential data leakage
## using the min and max from the trainset to do minmax on test set
age_min, age_max = X_train["age"].min(), X_train["age"].max()
X_train["age"] = (X_train["age"] - age_min) / (age_max - age_min)
X_test["age"] = (X_test["age"] - age_min) / (age_max - age_min)

BMI_min, BMI_max = X_train["BMI"].min(), X_train["BMI"].max()
X_train["BMI"] = (X_train["BMI"] - BMI_min) / (BMI_max - BMI_min)
X_test["BMI"] = (X_test["BMI"] - BMI_min) / (BMI_max - BMI_min)

## Random oversampling using random seed
## Define oversampling strategy: default, 'auto': equivalent to 'not majority'
oversample = RandomOverSampler(random_state=r_seed)
os_data_X, os_data_y = oversample.fit_resample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X, columns=columns)
os_data_y= pd.DataFrame(data=os_data_y, columns=['IMV_or_death'])


# define undersample strategy: “majority” will undersample the majority class determined by the class with the largest number of examples.
# undersample = RandomUnderSampler(sampling_strategy='majority', random_state=r_seed)
# us_data_X, us_data_y = undersample.fit_resample(X_train, y_train)
# us_data_X = pd.DataFrame(data=us_data_X, columns=columns)
# us_data_y= pd.DataFrame(data=us_data_y, columns=['IMV_or_death'])


# we can Check the numbers of our data
# print("length of oversampled data is ",len(os_data_X))
# print("Number of no hos in oversampled data",len(os_data_y[os_data_y['hospitalized_after_positive']==0]))
# print("Number of hos",len(os_data_y[os_data_y['hospitalized_after_positive']==1]))
# print("Proportion of no hos data in oversampled data is ",len(os_data_y[os_data_y['hospitalized_after_positive']==0])/len(os_data_X))
# print("Proportion of hos data in oversampled data is ",len(os_data_y[os_data_y['hospitalized_after_positive']==1])/len(os_data_X))

unique labels from y: [0 1]

In [0]:
IMV_death_result_df, os_LR_statmodel = multivariate_LR_model_result(os_data_X, os_data_y, X_train, y_train, opt_method)
display(IMV_death_result_df)

Maximum Likelihood optimization failed to converge. Check mle_retvals
Maximum Likelihood optimization failed to converge. Check mle_retvals

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
iteritems is deprecated and will be removed in a future version. Use .items instead.

Feature,Feature Importance,Odds ratio,lower,upper,Pvalue,Pvalue_FDR_corrected
age,0.9736801505805307,0.9487,0.924,0.974,0.0,0.0
BMI,3.526473380356116,-0.2848,-4.096,3.526,0.887,1.0
sex:male,0.27224276473847075,-0.2883,-0.304,-0.272,0.0,0.0
hypertension,0.14568833453685587,0.1225,0.099,0.146,0.0,0.0
diabetes (type 1+2),0.4876282280749658,0.4612,0.435,0.488,0.0,0.0
atrial fibrillation,0.6558609581178938,0.62,0.584,0.656,0.0,0.0
coronary artery disease,0.25657447803264954,0.221,0.185,0.257,0.0,0.0
heart failure,0.6773496757412416,0.6404,0.603,0.677,0.0,0.0
chronic kidney disease,0.7930422573580556,0.7602,0.727,0.793,0.0,0.0
COPD,0.7536902206579797,0.7145,0.675,0.754,0.0,0.0


In [0]:
from sklearn.metrics import roc_auc_score
LR_model_binaryclass_auroc = binary_class_roc_auc_score(y_test, os_LR_statmodel.predict(X_test))

print("Auroc on test data set: %0.5f" % (LR_model_binaryclass_auroc))

Auroc on test data set: 0.81402